In [1]:
# --- Dynamics: one subplot per γ (4 subplots in one row) ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

# Global font size slightly larger
mpl.rcParams["font.size"] = 12
mpl.rcParams["axes.titlesize"] = 13
mpl.rcParams["axes.labelsize"] = 12
mpl.rcParams["xtick.labelsize"] = 11
mpl.rcParams["ytick.labelsize"] = 11
mpl.rcParams["legend.fontsize"] = 11

In [38]:
# --- Parameters ---
N_firms = 3
T = 500
mutation_rate = 0.05
runs_per_setting = 10
betas = [0.0, 1.0, 2.0]
gammas = np.linspace(0, 2, 11)
ks = np.linspace(0, 2.0, 11)
epsilon = 0.1

# --- Profit function ---
def profit(a_i, a_j, gamma, beta, k):
    return gamma * a_i * (1 - a_j) + beta * a_i - k * a_i**2

# --- Analytical convergence ---
def analytical_convergence(gamma, k, beta=1.0, epsilon=0.1, T=1000, a0=0.1):
    # ÉVITER division par zéro
    denominator = 2 * max(k, 1e-8) + gamma  # k minimum 1e-8
    a_star = min(1.0, (gamma + beta) / denominator)
    t = np.arange(T)
    a_t = a_star + (a0 - a_star) * np.exp(-epsilon * t)
    return t, a_t, a_star

# --- Simulation (beta as parameter) ---
def simulate(gamma, k, beta=1.0):
    automation_levels = np.random.rand(N_firms) / 10
    history = []
    for t in range(T):
        new_levels = automation_levels.copy()
        profits = np.zeros(N_firms)
        for i in range(N_firms):
            a_j = np.mean(np.delete(automation_levels, i))
            profits[i] = profit(automation_levels[i], a_j, gamma, beta, k)
        for i in range(N_firms):
            j = np.random.choice(np.delete(np.arange(N_firms), i))
            pi_i = profits[i]
            pi_j = profits[j]
            p_adopt = 1 / (1 + np.exp(-(pi_j - pi_i)))
            if np.random.rand() < p_adopt:
                new_levels[i] = automation_levels[j]
            if np.random.rand() < mutation_rate:
                new_levels[i] = np.clip(
                    automation_levels[i] + np.random.normal(scale=0.1), 0, 1
                )
        automation_levels = new_levels
        history.append(np.mean(automation_levels))
    return history


In [ ]:
# --- Collect simulation results for all betas ---
all_records = []

for beta_value in betas:
    print(f"\n=== Simulating β={beta_value} ===")
    records = []
    for gamma in gammas:
        for k in ks:
            print(f"  γ={gamma:.2f}, k={k:.2f}...", end=" ")
            for run in range(runs_per_setting):
                history = simulate(gamma, k, beta=beta_value)
                for t, avg_auto in enumerate(history):
                    records.append({
                        "beta": beta_value,
                        "gamma": gamma,
                        "k": k,
                        "round": t,
                        "run": run,
                        "avg_automation": avg_auto,
                    })
            print(f"done ({runs_per_setting} runs)")
    all_records.extend(records)

df = pd.DataFrame(all_records)
stats = (
    df.groupby(["beta", "gamma", "k", "round"])["avg_automation"]
    .agg(["mean", "std"])
    .reset_index()
)


=== Simulating β=0.0 ===
  γ=0.00, k=0.00... done (10 runs)
  γ=0.00, k=0.20... done (10 runs)
  γ=0.00, k=0.40... done (10 runs)
  γ=0.00, k=0.60... done (10 runs)
  γ=0.00, k=0.80... done (10 runs)
  γ=0.00, k=1.00... done (10 runs)
  γ=0.00, k=1.20... done (10 runs)
  γ=0.00, k=1.40... done (10 runs)
  γ=0.00, k=1.60... done (10 runs)
  γ=0.00, k=1.80... done (10 runs)
  γ=0.00, k=2.00... done (10 runs)
  γ=0.20, k=0.00... done (10 runs)
  γ=0.20, k=0.20... done (10 runs)
  γ=0.20, k=0.40... done (10 runs)
  γ=0.20, k=0.60... done (10 runs)
  γ=0.20, k=0.80... done (10 runs)
  γ=0.20, k=1.00... done (10 runs)
  γ=0.20, k=1.20... done (10 runs)
  γ=0.20, k=1.40... done (10 runs)
  γ=0.20, k=1.60... done (10 runs)
  γ=0.20, k=1.80... done (10 runs)
  γ=0.20, k=2.00... done (10 runs)
  γ=0.40, k=0.00... done (10 runs)
  γ=0.40, k=0.20... done (10 runs)
  γ=0.40, k=0.40... done (10 runs)
  γ=0.40, k=0.60... done (10 runs)
  γ=0.40, k=0.80... done (10 runs)
  γ=0.40, k=1.00... done (10 

In [40]:
# --- 3 Heatmaps avec UNE SEULE colorbar parfaitement positionnée ---
final_stats = stats[stats["round"] == T - 1]

mpl.rcParams["font.size"] = 11
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

vmin, vmax = 0, 1

for idx, beta_value in enumerate(betas):
    ax = axes[idx]
    
    beta_data = final_stats[final_stats["beta"] == beta_value]
    pivot_heatmap = beta_data.pivot(index="k", columns="gamma", values="mean")
    
    sns.heatmap(
        pivot_heatmap,
        annot=False,
        cmap="RdYlBu_r",  # Rouge → Bleu
        vmin=vmin,
        vmax=vmax,
        ax=ax,
        xticklabels=[f"{g:.1f}" for g in gammas],
        yticklabels=[f"{k:.1f}" for k in ks],
        cbar=idx==2,  # SEULEMENT la dernière heatmap a une colorbar
        cbar_kws={'label': 'Final automation level'}
    )
    
    ax.set_title(rf"$\beta={beta_value}$", fontsize=13, pad=10)
    ax.set_xlabel(r"$\gamma$")
    if idx == 0:
        ax.set_ylabel(r"$k$")

plt.tight_layout()
plt.savefig("simulation_heatmap.pdf", dpi=300, bbox_inches="tight")
plt.close()


In [42]:
# --- Code GÉNÉRALISÉ (version finale optimisée) ---
betas_subset = [0.0, 1.0, 2.0]
gammas_subset = [0.0, 1.0, 2.0]
ks_subset = [0.0, 1.0, 2.0]

# Filtrage
filtered_stats = stats[
    (stats["beta"].isin(betas_subset)) &
    (stats["gamma"].isin(gammas_subset)) &
    (stats["k"].isin(ks_subset))
]
print(f"Données filtrées: {len(filtered_stats)} lignes")

# Grille 3×3
n_betas, n_gammas = len(betas_subset), len(gammas_subset)
fig, axes = plt.subplots(n_betas, n_gammas, figsize=(18, 12), sharex=True, sharey=True)
axes = axes.ravel()

colors = plt.cm.viridis(np.linspace(0, 1, len(ks_subset)))

for row, beta in enumerate(betas_subset):
    for col, gamma in enumerate(gammas_subset):
        idx = row * n_gammas + col
        ax = axes[idx]
        
        beta_gamma_data = filtered_stats[
            (filtered_stats["beta"] == beta) & (filtered_stats["gamma"] == gamma)
        ]
        
        # Style
        for spine in ax.spines.values():
            spine.set_visible(True)
            spine.set_linewidth(1.2)
        ax.grid(True, alpha=0.3, linestyle=':')
        
        # k loop
        for color_idx, k in enumerate(ks_subset):
            color = colors[color_idx]
            
            # SIMU
            gk = beta_gamma_data[beta_gamma_data["k"] == k]
            if len(gk) > 0:
                ax.plot(gk["round"], gk["mean"], color=color, linewidth=2.5)
                ax.fill_between(gk["round"], gk["mean"] - gk["std"]/2,
                               gk["mean"] + gk["std"]/2, color=color, alpha=0.2)
            
            # THÉORIQUE
            t_anal, a_anal, _ = analytical_convergence(gamma, k, beta=beta, T=T)
            ax.plot(t_anal[:T], a_anal[:T], "--", color=color, linewidth=1.8)
        
        ax.set_title(rf"$\beta={beta}$, $\gamma={gamma:.1f}$", fontsize=13, fontweight='bold')
        
        if row == n_betas - 1:
            ax.set_xlabel("Iteration (quarters)", fontsize=12)
        if col == 0:
            ax.set_ylabel("Avg. automation", fontsize=12)

# Légende propre

# LÉGENDE CORRIGÉE
handles = []
labels = []
for i, k in enumerate(ks_subset):
    handles.append(plt.Line2D([0], [0], color=colors[i], lw=2.5, label=f'k={k:.1f} (sim)'))
    labels.append(f'k={k:.1f} (sim)')
    handles.append(plt.Line2D([0], [0], color=colors[i], lw=1.8, ls='--', label=f'k={k:.1f} (th)'))
    labels.append(f'k={k:.1f} (th)')

fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.02), 
           ncol=len(ks_subset), fontsize=11)

plt.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.savefig("simulation_dynamics_3x3.pdf", dpi=300, bbox_inches="tight")
plt.close()

print("✅ Figure sauvée SANS ERREUR !")

Données filtrées: 13500 lignes
✅ Figure sauvée SANS ERREUR !
